In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Colab\ Notebooks/HW5

/content/gdrive/MyDrive/Colab Notebooks/HW5


In [3]:
# !pip install numpy==1.16.2
import numpy as np
print(np.__version__)
# !pip install torch
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
!pip install torchsummary 
from torchsummary import summary
from torch.autograd import Variable
!pip install pytorch_ssim
import pytorch_ssim

import math
from math import exp
import time
import matplotlib.pyplot as plt
import csv
import os
from os import path
import gc
import glob
from PIL import Image

1.19.5
  Created wheel for pytorch-ssim: filename=pytorch_ssim-0.1-py3-none-any.whl size=2026 sha256=1bb7e3241b2321720f980b661fc08a72ce3c7029fdaa5cb059220bebe24438d6
  Stored in directory: /root/.cache/pip/wheels/ee/20/09/ebf5e58bdf2560c760074cd140b7f7b0c882e216feabf1ae30
Successfully built pytorch-ssim


In [4]:
cuda = torch.cuda.is_available()
print(cuda)
num_workers = 2 if cuda else 0

True


In [5]:
transform = transforms.Compose([          
            transforms.ToTensor(),
            ])

In [6]:
class MyDataset(Dataset):
  def __init__(self, image_in_folder, image_out_folder=None, transform=None):
    self.image_in_folder = image_in_folder
    self.image_in_paths = sorted(os.listdir(image_in_folder))
    if image_out_folder:
      self.image_out_folder = image_out_folder
      self.image_out_paths = sorted(os.listdir(image_out_folder))
    else:
      self.image_out_folder = None
    self.transform = transform
            
  def __getitem__(self, index):
    x = Image.open(self.image_in_folder + self.image_in_paths[index])
    x = x.convert('RGB')
    if self.image_out_folder:
        y = Image.open(self.image_out_folder + self.image_out_paths[index])
        y = y.convert('RGB')
    else:
        y = self.image_in_paths[index]
    if self.transform is not None:
        x = self.transform(x)
        if self.image_out_folder is not None:
            y = self.transform(y)
            i, j, h, w = transforms.RandomCrop.get_params(x, output_size=(min(x.size(1), 200), min(x.size(2), 200)))
            x = transforms.functional.crop(x, i, j, h, w)
            y = transforms.functional.crop(y, i*4, j*4, h*4, w*4)
    return x, y

  def __len__(self):
    return len(self.image_in_paths)

In [27]:
train_x = 'HW5_IDLS21/HW5_train_val/DIV2K_train_LR_bicubic/X4/'
train_y = 'HW5_IDLS21/HW5_train_val/DIV2K_train_HR/'
train_dataset = MyDataset(train_x, train_y, transform)
train_loader_args = dict(shuffle=True, batch_size=1, num_workers=num_workers, pin_memory=True, drop_last=True) \
                    if cuda else dict(shuffle=True, batch_size=1)
train_loader = DataLoader(train_dataset, **train_loader_args)
del train_x, train_y

val_x = 'HW5_IDLS21/HW5_train_val/DIV2K_valid_LR_bicubic/X4/'
val_y = 'HW5_IDLS21/HW5_train_val/DIV2K_valid_HR/'
val_dataset = MyDataset(val_x, val_y, transform)
val_loader_args = dict(shuffle=False, batch_size=1, num_workers=num_workers, pin_memory=True) \
                    if cuda else dict(shuffle=False, batch_size=1)
val_loader = DataLoader(val_dataset, **val_loader_args)
del val_x, val_y

# test_x_large = 'HW5_IDLS21/HW5_test/large_test/'
test_x_large = 'HW5_IDLS21/HW5_test/structures/'
test_dataset = MyDataset(test_x_large, None, transform)
test_loader_args = dict(shuffle=False, batch_size=1, num_workers=num_workers, pin_memory=True) \
                    if cuda else dict(shuffle=False, batch_size=1)
test_loader = DataLoader(test_dataset, **test_loader_args)
del test_x_large

In [8]:
print(test_dataset.__getitem__(0)[0].size(), test_dataset.__getitem__(0)[1])

torch.Size([3, 120, 125]) baboon.png


In [9]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super(Discriminator, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(64, 64, 3, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(64, 128, 3, 1, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(128, 128, 3, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(128, 256, 3, 1, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(256, 256, 3, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(256, 512, 3, 1, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(512, 512, 3, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, True)
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(512, 1024, kernel_size=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(1024, 1, kernel_size=1)
        )

        self._initialize_weights()

    def forward(self, x):
        out = self.features(x)
        out = self.classifier(out)
        out = nn. Sigmoid()(out.view(out.size(0)))
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.normal_(m.weight, 1.0, 0.02)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, channels = 64):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += x

        return out

In [11]:
class SubpixelConvolutionLayer(nn.Module):
    def __init__(self, channels = 64):
        super(SubpixelConvolutionLayer, self).__init__()
        self.conv = nn.Conv2d(channels, channels * 4, 3, 1, 1)
        self.pixel_shuffle = nn.PixelShuffle(2)
        self.prelu = nn.PReLU()

    def forward(self, x):
        out = self.conv(x)
        out = self.pixel_shuffle(out)
        out = self.prelu(out)

        return out

In [12]:
class Generator(nn.Module):
    def __init__(self) -> None:
        super(Generator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 9, 1, 4),
            nn.PReLU()
        )

        trunk = []
        for _ in range(16):
            trunk += [ResidualBlock(64)]
        self.trunk = nn.Sequential(*trunk)

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1, bias=False),
            nn.BatchNorm2d(64)
        )

        subpixel_conv = []
        for _ in range(2):
            subpixel_conv += [SubpixelConvolutionLayer(64)]
        self.subpixel_conv = nn.Sequential(*subpixel_conv)

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 3, 9, 1, 4),
            nn.Tanh()
        )

        self._initialize_weights()

    def forward(self, x):
        conv1 = self.conv1(x)
        trunk = self.trunk(conv1)
        conv2 = self.conv2(trunk)
        out = conv1 + conv2
        out = self.subpixel_conv(out)
        out = self.conv3(out)
        out = (out + 1) / 2
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.normal_(m.weight, 1.0, 0.02)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)


In [13]:
class ContentLoss(nn.Module):
    def __init__(self):
        super(ContentLoss, self).__init__()
        vgg19 = models.vgg19(pretrained=True).eval()
        self.model = nn.Sequential(*list(vgg19.features.children())[:36])
        for _, parameters in self.model.named_parameters():
            parameters.requires_grad = False
        self.mse_loss = nn.MSELoss()

    def forward(self, source, target):
        vgg_loss = self.mse_loss(self.model(source), self.model(target))
        mse_loss = self.mse_loss(source, target)
        loss = mse_loss + 0.006 * vgg_loss
        return loss

In [14]:
netD = Discriminator()
print(netD)
netD.cuda()
summary(netD.cuda(), (3, 128, 128))

Discriminator(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.

In [15]:
netG = Generator()
print(netG)
netG.cuda()
summary(netG.cuda(), (3, 128, 128))

Generator(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (1): PReLU(num_parameters=1)
  )
  (trunk): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=

In [16]:
# Calculate the one-dimensional Gaussian distribution vector
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()
 
 
 # Create a Gaussian kernel, obtained by matrix multiplication of two one-dimensional Gaussian distribution vectors
 # You can set the channel parameter to expand to 3 channels
def create_window(window_size, channel=1):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window
 
 
 # Calculate SSIM
 # Use the formula of SSIM directly, but when calculating the average value, instead of directly calculating the pixel average value, normalized Gaussian kernel convolution is used instead.
 # The formula Var(X)=E[X^2]-E[X]^2, cov(X,Y)=E[XY]-E[X]E[Y] is used when calculating variance and covariance .
 # As mentioned earlier, the above expectation operation is replaced by Gaussian kernel convolution.
def ssim(img1, img2, window_size=11, window=None, size_average=True, full=False, val_range=None):
    # Value range can be different from 255. Other common ranges are 1 (sigmoid) and 2 (tanh).
    if val_range is None:
        if torch.max(img1) > 128:
            max_val = 255
        else:
            max_val = 1
 
        if torch.min(img1) < -0.5:
            min_val = -1
        else:
            min_val = 0
        L = max_val - min_val
    else:
        L = val_range
 
    padd = 0
    (_, channel, height, width) = img1.size()
    if window is None:
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channel).to(img1.device)
 
    mu1 = F.conv2d(img1, window, padding=padd, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padd, groups=channel)
 
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2
 
    sigma1_sq = F.conv2d(img1 * img1, window, padding=padd, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padd, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padd, groups=channel) - mu1_mu2
 
    C1 = (0.01 * L) ** 2
    C2 = (0.03 * L) ** 2
 
    v1 = 2.0 * sigma12 + C2
    v2 = sigma1_sq + sigma2_sq + C2
    cs = torch.mean(v1 / v2)  # contrast sensitivity
 
    ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)
 
    if size_average:
        ret = ssim_map.mean()
    else:
        ret = ssim_map.mean(1).mean(1).mean(1)
 
    if full:
        return ret, cs
    return ret
 
 
 
# Classes to re-use window
class SSIM(torch.nn.Module):
    def __init__(self, window_size=11, size_average=True, val_range=None):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.val_range = val_range
 
        # Assume 1 channel for SSIM
        self.channel = 1
        self.window = create_window(window_size)
 
    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()
 
        if channel == self.channel and self.window.dtype == img1.dtype:
            window = self.window
        else:
            window = create_window(self.window_size, channel).to(img1.device).type(img1.dtype)
            self.window = window
            self.channel = channel
 
        return ssim(img1, img2, window=window, window_size=self.window_size, size_average=self.size_average)

In [17]:
def mypsnr(img1, img2):
    max_pixel = 1.0
    mse = np.mean((img1 - img2)**2)
    psnr = 20 * math.log(max_pixel / math.sqrt(mse), 10)
    
    return psnr
    
def myssim(img1, img2):
    #convert to tensor
    img1 = torch.tensor(img1).cuda()
    img2 = torch.tensor(img2).cuda()

    #convert to variable
    img1 = Variable(img1,  requires_grad=False)
    img2 = Variable(img2, requires_grad = True)
    
    # ssim_value = pytorch_ssim.ssim(img1, img2)
    ssim_value = ssim(img1, img2)
    
    return ssim_value.item()

In [18]:
def train_epoch(netG, netD, train_loader, content_criterion, optimiserG, optimiserD, results, epoch):
  print("Training...")
  netG.train()
  netD.train()
  running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}

  start_time = time.time()

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.cuda(), target.cuda()
    g_update_first = True
    batch_size = data.size(0)
    running_results['batch_sizes'] += batch_size
    
    netD.zero_grad()
    fake_img = netG(data)
    fake_out = netD(fake_img).mean()
    real_out = netD(target).mean()
    d_loss = 1 - real_out + fake_out
    d_loss.backward()
    optimiserD.step()

    if epoch > 2:
      netG.zero_grad()
      fake_img = netG(data)
      fake_out = netD(fake_img).mean()
      g_loss = content_criterion(fake_img, target) + 0.001 * (torch.mean(1 - fake_out))
      g_loss.backward()
      optimiserG.step()
    else:
      g_loss = None

    # loss for current batch before optimization 
    fake_img = netG(data)
    fake_out = netD(fake_img).mean()
    
    if g_loss:
      running_results['g_loss'] += g_loss.item() * batch_size
    running_results['d_loss'] += d_loss.item() * batch_size
    running_results['d_score'] += real_out.item() * batch_size
    running_results['g_score'] += fake_out.item() * batch_size
    
    del fake_img
    del fake_out
    del real_out
    torch.cuda.empty_cache()
    if (batch_idx+1) % 200 == 0:
      print("\tBatch: %d Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f" % (
          batch_idx + 1,
          running_results['d_loss'] / running_results['batch_sizes'],
          running_results['g_loss'] / running_results['batch_sizes'],
          running_results['d_score'] / running_results['batch_sizes'],
          running_results['g_score'] / running_results['batch_sizes']))
      
  end_time = time.time()

  print("Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f Time: %.4f" % (
      running_results['d_loss'] / running_results['batch_sizes'],
      running_results['g_loss'] / running_results['batch_sizes'],
      running_results['d_score'] / running_results['batch_sizes'],
      running_results['g_score'] / running_results['batch_sizes'],
      end_time - start_time))
  
  results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
  results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
  results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
  results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])

In [19]:
def val_epoch(netG, netD, val_loader, results):
    print("Validating...")
    netG.eval()
    start_time = time.time()

    with torch.no_grad():
      val_results = {'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
      for batch_idx, (data, target) in enumerate(val_loader):
        data, target = data.cuda(), target.cuda()
        batch_size = data.size(0)
        val_results['batch_sizes'] += batch_size
        fake_img = netG(data)
        t,o = target.cpu().numpy(), fake_img.cpu().numpy()

        val_results['ssim'] += myssim(t, o)
        val_results['psnr'] += mypsnr(t, o)

        del fake_img
        torch.cuda.empty_cache()
    
    end_time = time.time()
    
    print("PSNR: %.4f dB SSIM: %.4f Time: %.4f" % (
          val_results['psnr']/val_results['batch_sizes'], 
          val_results['ssim']/val_results['batch_sizes'],
          end_time - start_time))
    
    results['psnr'].append(val_results['psnr']/val_results['batch_sizes'])
    results['ssim'].append(val_results['ssim']/val_results['batch_sizes'])

In [20]:
n_epochs = 200
model_no = 6

learningRate = 0.0005
weightDecay = 1e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

netG = Generator()
netD = Discriminator()

netG.to(device)
netD.to(device)

# netG  # convert to half precision
# for layer in netG.modules():
#   if isinstance(layer, nn.BatchNorm2d):
#     layer.float()

# netD  # convert to half precision
# for layer in netD.modules():
#   if isinstance(layer, nn.BatchNorm2d):
#     layer.float()

def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        # m.bias.data.fill_(0.01)

netG.apply(init_weights)
netD.apply(init_weights)

content_criterion = ContentLoss()
content_criterion.to(device)

optimiserG = optim.Adam(netG.parameters(), lr=learningRate, weight_decay=weightDecay)
optimiserD = optim.Adam(netD.parameters(), lr=learningRate, weight_decay=weightDecay)

schedulerG = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiserG, mode='min', factor=0.7, patience=5)
schedulerD = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiserD, mode='min', factor=0.7, patience=5)

results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}

torch.cuda.empty_cache()

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
for i in range(n_epochs):
    print("Epoch ", i)

    if path.exists('SRGAN/model_'+ str(model_no) + '_' + str(i)) and i<4:
      checkpoint = torch.load('SRGAN/model_' + str(model_no) + '_' + str(i))
      netG.load_state_dict(checkpoint['netG_state_dict'])
      netD.load_state_dict(checkpoint['netD_state_dict'])
      optimiserG.load_state_dict(checkpoint['optimiserG_state_dict'])
      optimiserD.load_state_dict(checkpoint['optimiserD_state_dict'])
      results = checkpoint['results']
      continue

    print("LRs: ", optimiserG.param_groups[0]['lr'], optimiserD.param_groups[0]['lr'])
    
    train_epoch(netG, netD, train_loader, content_criterion, optimiserG, optimiserD, results, i)
    val_epoch(netG, netD, val_loader, results)
    
    torch.save({
            'epoch': i,
            'netG_state_dict': netG.state_dict(),
            'netD_state_dict': netD.state_dict(),
            'optimiserG_state_dict': optimiserG.state_dict(),
            'optimiserD_state_dict': optimiserD.state_dict(),
            'results': results,
            }, 'SRGAN/model_' + str(model_no) + '_' + str(i))

In [21]:
checkpoint = torch.load('SRGAN/model_6_34')
netG.load_state_dict(checkpoint['netG_state_dict'])

results = checkpoint['results']      
PSNR = results['psnr']
SSIM = results['ssim']

print(PSNR)  
print(SSIM)
print(np.argmax(PSNR), np.max(PSNR))
print(np.argmax(SSIM), np.max(SSIM))
print(len(val_dataset))
for i in range(35):
  if PSNR[i]/100>26:
    print(i, PSNR[i]/100, SSIM[i]/100)

[1030.5273474642831, 1027.5995542532137, 1058.5124003434846, 2203.493038737403, 2359.4276393697887, 2504.3242278720636, 2487.444830609717, 2444.485887764993, 2395.966601029158, 2512.1139754503124, 2246.1774055340534, 2597.0810326571723, 2533.2055271415425, 2437.3428699436313, 2588.3552079343053, 2672.441981853846, 2622.5102374884646, 2633.2591295534494, 2648.649254338814, 2656.977963741113, 2461.2712855144937, 2675.6570692382215, 2678.6784230218923, 2559.7094076021776, 2648.9757784152434, 2628.448673179513, 2632.8632968451448, 2585.3721527001817, 2701.7161700379247, 2709.160020957549, 2708.0177240609796, 2691.7816380470776, 2518.239474090298, 2690.083472183182, 2617.4244692101406]
[4.728824796155095, 4.4555251728743315, 5.332370828371495, 62.24700404703617, 67.59241497516632, 69.19052425026894, 69.70718145370483, 69.51358643174171, 71.3279781639576, 72.8458186686039, 67.78799712657928, 73.2539653480053, 73.35457733273506, 72.11695542931557, 72.12467968463898, 75.25061735510826, 72.3614

In [28]:
def test_epoch(netG, test_loader):
    print("Testing...")
    netG.eval()
    start_time = time.time()

    with torch.no_grad():
      for batch_idx, (data, label) in enumerate(test_loader):
        data = data.cuda()
        batch_size = data.size(0)
        fake_img = netG(data)
        # np.save("Test_5_111/" + label[0][:-4], fake_img[0].cpu().numpy().transpose(1,2,0))
        save_image(fake_img[0].cpu(), "Test_SRGAN_6_29_Structures/" + label[0])
        del fake_img
        torch.cuda.empty_cache()
    
    end_time = time.time()

In [22]:
def test_epoch(netG, test_loader):
    print("Testing...")
    netG.eval()
    start_time = time.time()

    with torch.no_grad():
      for batch_idx, (data, label) in enumerate(test_loader):
        data = data.cuda()
        size = (data.size(2)*4, data.size(3)*4)
        data_size = (200, int(data.size(3)/data.size(2)*200))
        if data.size(3) > data.size(2):
          data_size = (int(data.size(2)/data.size(3)*200), 200)
        print(data.size())
        data = F.interpolate(data, data_size)
        batch_size = data.size(0)
        fake_img = netG(data)
        fake_img = F.interpolate(fake_img, size)
        # np.save("Test_5_111/" + label[0][:-4], fake_img[0].cpu().numpy().transpose(1,2,0))
        save_image(fake_img[0].cpu(), "Test_SRGAN_6_29_Scaled200_4_Valid/" + label[0])
        del fake_img
        torch.cuda.empty_cache()
    
    end_time = time.time()

In [23]:
def test_epoch(netG, test_loader):
    print("Testing...")
    netG.eval()
    start_time = time.time()

    with torch.no_grad():
      for batch_idx, (data, label) in enumerate(test_loader):
        data = data.cuda()
        size = (data.size(2)*4, data.size(3)*4)
        data_size = (200, int(data.size(3)/data.size(2)*200))
        if data.size(3) > data.size(2):
          data_size = (int(data.size(2)/data.size(3)*200), 200)
        print(data_size)
        data = F.interpolate(data, data_size, mode='bilinear', align_corners=True)
        batch_size = data.size(0)
        fake_img = netG(data)
        fake_img = F.interpolate(fake_img, size, mode='bilinear', align_corners=True)
        # np.save("Test_5_111/" + label[0][:-4], fake_img[0].cpu().numpy().transpose(1,2,0))
        save_image(fake_img[0].cpu(), "Test_SRGAN_6_29_Scaled200_5/" + label[0])
        del fake_img
        torch.cuda.empty_cache()
    
    end_time = time.time()

In [29]:
checkpoint = torch.load('SRGAN/model_6_29')
netG.load_state_dict(checkpoint['netG_state_dict'])
!mkdir Test_SRGAN_6_29_Structures
test_epoch(netG, test_loader)

Testing...
